In [0]:
# -*- coding: utf-8 -*-
import dataiku
import datetime
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
from dataiku import pandasutils as pdu

In [0]:
# Connect to folder
sample_batches = dataiku.Folder("zY3ZURve")
sample_batches_info = sample_batches.get_info()

# Path in folder
data_folder = dataiku.get_custom_variables()["batches_path"]

In [0]:
# Model
seed = 112233
np.random.seed(seed)
reference_time = datetime.datetime(2024, 1, 1)

def get_features(current_time: datetime.datetime, fixed_delta:float, size:tuple = (1000,1)) -> pd.DataFrame:
    # Time
    time_delta = current_time - reference_time
    timestamps = pd.date_range(current_time, periods=size[0], freq="s")
    
    # Trend increses in time to saturation
    long_trend = min((0.008 * time_delta.total_seconds() / (24 * 60 * 60)), 1) * np.ones(size)
    
    # Intraday
    short_trend = np.ones(size)
    
    # Price delta
    price_delta = fixed_delta * np.ones(size)
    
    # Random noise
    noise_a = np.random.normal(0, 0.1, size)
    
    df = pd.DataFrame(
        {
            "timestamps": timestamps,
            "long_trend": long_trend,
            "short_trend": short_trend,
            "price_delta": price_delta,
        }
    )
    
    return df

def get_outcome(features: pd.DataFrame, price_delta) -> pd.Series:
    # Price dependency
    price_sensitivity = - 0.2 / (1 + np.exp(-features['price_delta']))
    
    return 

In [0]:
# Produce the data and save to the folder
current_time = datetime.datetime.now()
print(current_time)
time_delta = (current_time - reference_time)
print(time_delta.days)
size = (1000, 1)
timestamp = pd.date_range(current_time, periods=size[0], freq="s")
print(timestamp)

In [0]:
# Produce features and the 
features = get_features_initial(current_time)
features['purchased'] = get_outcome(features)